<a href="https://colab.research.google.com/github/rzl-ds/gu511_hw/blob/master/hw09.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg"/>
</a>

# Exercises due by EOD 2020.11.13

## goal

in this homework assignment we will learn how to work with the `rds` database service and `redshift`, as well as handle our first(ish) `git` CONFLICT

## method of delivery

*as mentioned in our first lecture, the method of delivery may change from assignment to assignment. we will include this section in every assignment to provide an overview of how we expect homework results to be submitted, and to provide background notes or explanations for "new" delivery concepts or methods.*

this week you will be submitting the results of your homework in a few ays: via your `github` repo, via upload to a shared `s3` homework bucket, and via record insertion into shared databases.

NO submission email

summary:

| exercise | deliverable | method of delivery | points |
|----------|-------------|--------------------|--------|
| 1 | a record in a shared `postgres` database | via an `INSERT` statement | 7 |
| 2 | none | none | 4 |
| 3 | none | none | 4 |
| 4 | a file `bulk_insert_times.csv` | uploaded to your `s3` homework bucket | 10 |
| 5 | none | none | 2 |
| 6 | none | none | 2 |
| 7 | none | none | 5 |
| 8 | a file named `create_trainpositions.sql` | uploaded to your `s3` homework submission bucket | 5 |
| 9 | a file named `copy_trainpositions.sql` | uploaded to your `s3` homework submission bucket | 5 |
| 10 | a conflict-resolving `merge` commit | pushed to your shared `github` repo | 6 |

total points: 50

<div style="border: 1px solid lightgrey;">

# exercise 1: connect and write to a `postgres` database

we have set up a shared `postgres` database with the following connection details:

| parameter  | value |
|------------|-------------------------------------------------------------|
| `host`     | `gu511-shared-hw-postgres-2020.cdmknaubrmaw.us-east-1.rds.amazonaws.com` |
| `port`     | `5432` |
| `username` | `gu511` |
| `db`       | `postgres` |
| `password` | emailed to you |

we would like for you to install the `psql` client (not the `postgres` database) on your `ec2 ubuntu` instances, use it to connect to this shared database, and `insert` a value into a shared table.

## 1.1: install `psql`

on your `ec2 ubuntu` instances, install `psql` using the command

```bash
sudo apt install postgresql-client
```

you can verify that your installation worked be checking

```bash
which psql
```

and receiving an output that is the file path to the `psql` executible

## 1.2: connect to the database

using the values we provided in the table above, the course notes, and `man psql`, figure out how to connect to our shared database with the `psql` command

## 1.3: `insert` a new record

the following command will insert a record into a shared database we've created called `gu511_shared_table`. the table has two columns:

1. `guid`: your georgetown user id
2. `message`: a test message to verify you were able to successfully insert your record

once you've successfully connect to the `postgres` database, insert a record with the following query (replacing `MY_GUID` and `MY_MESSAGE` with real values, of course):

```sql
INSERT INTO gu511_shared_table (guid, message) VALUES ('MY_GUID', 'MY_MESSAGE');
```

*note*: the single quotes in `'MY_GUID', 'MY_MESSAGE'` are required. they indicate that the values being passed in are strings, which is the datatype of those two coumns in our table


##### the submission is the record in the shared database -- we will check results by running `SELECT * FROM gu511_shared_table;;`

<div style="border: 1px solid lightgrey;">

# exercise 2: creating an `rds` postgres instance

**note: this is not significantly different from what we did in class; if you already did that you do not need to make a new one for this purpose**

create your own `postgres` `rds` instance using the `aws` `rds` service. select the "Easy Create" method and update

1. engine type: `postgresql`
1. db instance size: "Free tier"
1. db instance identifier: whatever you want
1. master password

after it starts up,

1. edit the security group to allow `postgres` traffic (TCP on port 5432) from your `ec2` instance's **private** ip address.

after it successfully starts up, be sure to capture the hostname, port, database name, master user name, and master user password.

##### there is nothing to submit for this assignment

<div style="border: 1px solid lightgrey;">

# exercise 3: creating an `ec2` server with `postgres` on it

choose one of the following two options (regular (3.1) vs. advanced (3.2))

## 3.1: use an `ami`

I have already done all the steps in the second "from scratch" option below and saved them as an `ami` that I have shared with you.

create a new `ec2` instance with `postgres` pre-installed and configured by

1. going to the `ec2` service
1. clicking on `ami` in the left menu
1. change "Owned by me" to "Private Images" in the dropdown next to the search bar
1. you should see an `ami` with `ami-id = ami-0922deee48c98a016` and name `gu511-dinky-postgres`

select that `ami` and click "Launch". the only thing you should have to configure is the `ssh` key.

test that this all worked by

1. log in via `ssh`
    1. note: user name on this image is `ec2-user`
    1. `ssh -i [path to your pem key here] ec2-user@[your ec2 public dns here]`
1. start `postgres` service with `sudo service postgresql start`
1. connect to that `postgres` service with `psql --user postgres`
    1. quit by entering `\q`
    
##### there is nothing to submit for this assignment

## 3.2: from scratch [ADVANCED]


the goal of this exercise is to replicate what `rds` does for us: create an `ec2` instance and install the `postgres` service. let's start by creating that server.

### 3.2.1: create the `ec2` server

first, using the `aws` `ec2` service, create a new `ec2` instance that is as close as possible to the underlying `ec2` instance of the `postgres` database instance we created in the previous problem  (that is, it should have the following properties):
    
+ `ami`: Amazon Linux 2 AMI (HVM), SSD Volume Type 64 bit (free tier)
+ instance type: `t2.micro`
+ storage: set the type to SSD (`gp2`) and the size of the storage to 20 GB

with that done, let's go about installing and configuring `postgres`. 

### 3.2.2: installing `postgres`

once that server starts up, `ssh` into it (note: for the `amazon linux ami` the username is `ec2-user`). let's start by installing `postgres` with the following commands:

```bash
sudo yum install postgresql postgresql-server postgresql-devel postgresql-contrib postgresql-docs
sudo postgresql-setup initdb
sudo service postgresql start
sudo vim /var/lib/pgsql/data/pg_hba.conf
```

### 3.2.3: configuring `postgres`

oh fun -- remember `vim`?!

we need to tell the `postgres` service to allow you to log in as the `postgres` user (which is technically considered bad practice, but let's do it for fun anyway). toward the bottom of the file you just opened in `vim` there is a pair of lines that look like the following:

```conf
# "local" is for Unix domain socket connections only
local   all             all                                     peer
```

this config line is saying that

+ for people logged on to your machine (`local`)
+ for all databases (the first `all`)
+ and for all postgres database service users (the second `all`)
+ use `peer` authentication
    + in `peer` authentication, the `postgres` service assumes that the local `linux` username and `postgres` user name are meant to be the same -- e.g. it would assume you want to log in as database user `ec2-user`

we will loosen up those restrictions by changing the `peer` authentication method to `trust`, which allows anyone who has made it onto our `ec2` instance (`local`) to view any database as any user (`all` and `all`).

```conf
# "local" is for Unix domain socket connections only
local   all             all                                     trust
```

edit this file to change `peer` to `trust`

### 3.2.4: verifying our install

once this edit has been made, you can finally start the server and log in to your own `postgres` database server:

```bash
sudo service postgresql restart

# only one user exists right now: postgres
psql -U postgres
```

the output of that should just be a `psql` prompt:

```
psql (9.2.24)
Type "help" for help.

postgres=# 
```

you're good! feel free to exit that `psql` shell with `\q`

### 3.2.5: installing `psycopg2`

one last thing: let's install `psycopg2`. execute the following:

```bash
cd /tmp
wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
# answer "yes" to all questions
bash Miniconda3-latest-Linux-x86_64.sh

cd
conda install psycopg2
```

##### there is nothing to submit for this assignment

<div style="border: 1px solid lightgrey;">

# exercise 4: comparing bulk `csv` insert methods

there are a few different ways to perform inserts of many records, and they have pretty drastically different performance. the fastest way to insert large quantities of data into a database is *usually* by utilizing a proprietary bulk insert command. for example, in

+ `ms sql`: [a `BULK INSERT` command](https://docs.microsoft.com/en-us/sql/t-sql/statements/bulk-insert-transact-sql)
+ `mysql`: [the `LOAD DATA INFILE` command](https://dev.mysql.com/doc/refman/5.6/en/load-data.html)
+ `oracle`: using a special oracle tool [`sql*loader`](https://docs.oracle.com/cd/B19306_01/server.102/b14215/ldr_concepts.htm) to write a "control" file (to outline the import steps) and import the datafile (this is what you pay hundreds of thousands of dollars for)

`postgres` has implemented both a standard [`sql COPY` command](https://www.postgresql.org/docs/current/static/sql-copy.html) and a `psql` [`\copy` meta command](https://www.postgresql.org/docs/9.2/static/app-psql.html#APP-PSQL-META-COMMANDS-COPY) for this task.

the former (`sql COPY` command) can only be executed by users with appropriate permissions, and *must* be done with a local file. since we are using an unaccessbile `ec2` instance, we are out of luck on that count.

the latter (`\copy psql` command) can be executed by a non-admin user, and can be run remotely. the drawback: it is executed on the client side, so we have to send the entire inserted file over the wire.

you now have two `postgres` databases at your disposal:

1. your `rds` instance
1. an `ec2` instance with `postgres` manually installed on it

let's try out three different copy options to see how considerable the speed differences are. first, though, we need to do some setup.

## 4.1: prepping

you will be performing this exercise from the `ec2` server from the previous exercise (the one you created which has `postgres` running on it). `ssh` into that `ec2` server and take care of a few setup options.

### 4.1.1: getting data

download the bulk `csv` of `wmata` train position data which we intend to upload to our various `postgres` servers:

```bash
cd /tmp
wget https://s3.amazonaws.com/shared.rzl.gu511.com/train_positions/train_positions.csv
chmod a+r train_positions.csv
cd ~
```

### 4.1.2: getting the `python` code

I've written a short snippet of `python` code to

1. mark the starting time
1. create a connection to your `rds` database
1. create a csv dictreader to read lines from `train_positions.csv`
1. iterate through those csv lines and insert them in batches
1. mark the ending time
1. print the total time it took to insert the records to the screen

download it to your `ec2` instance and review it:

```bash
cd ~
wget https://s3.amazonaws.com/shared.rzl.gu511.com/train_positions/bulkinsert.py
```

### 4.1.3: creating an empty table on both dbs

you can log into your `ec2` `postgres` server with

```bash
psql -U postgres
```

and you can log into your `rds` server with

```bash
psql --host YOUR_RDS_ENDPOINT --port YOUR_RDS_PORT --dbname YOUR_DB_NAME --user YOUR_MASTER_USER_NAME
```

log into each and execute the following `sql` code:

```sql
CREATE TABLE train_positions (   
    carcount real                
    , circuitid real             
    , destinationstationcode text
    , directionnum real          
    , linecode text              
    , secondsatlocation real     
    , servicetype text           
    , trainid text               
    , timestamp timestamp        
);                               
```

you should get

```bash
CREATE TABLE
```

as a result of that command on either server. anything else may be an error.

create this empty table on both servers!

## 4.2: inserting in batches via `python` (remote)

the first way we will attempt to insert these records is to use the `psycopg2` library to `INSERT` records in batches of 100 at a time. 

assuming your working directory is the one in which you downloaded the file `bulkinsert.py` above, run the following from your `ec2` server's terminal:

```bash
python bulkinsert.py --host YOUR_RDS_ENDPOINT --port YOUR_RDS_PORT --dbname YOUR_DB_NAME --user YOUR_MASTER_USER_NAME --fcsv /tmp/train_positions.csv
```

for example, I successfully ran

```bash
python bulkinsert.py --host trainposdb.cdmknaubrmaw.us-east-1.rds.amazonaws.com --port 5432 --dbname postgres --user postgres --fcsv /tmp/train_positions.csv
```

note how long this process took (it will take minutes, not seconds or hours).

## 4.3: `psql` `\copy` command (remote)

what the previous command did was something pretty common: given a `csv` of records that matches the schema of the table we created, load all of the records. as it happens, this is so common that most relational databases have implemented shortcuts for doing exactly that.

one such shortcut is the `psql` command `\copy`. let's use the `\copy` command to copy this *local* `csv` file to our *remote* `rds` database.

on your `ec2` server, open a `psql` shell connection to your `rds` instance:

```bash
psql --host YOUR_RDS_ENDPOINT --port YOUR_RDS_PORT --dbname YOUR_DB_NAME --user YOUR_MASTER_USER_NAME
```

once in, execute the following `psql` command to turn on query timing (this will measure how long a query takes for us):

```sql
\timing
```

now, execute the `\copy` command:

```sql
\copy train_positions from /tmp/train_positions.csv with delimiter as ',' null as '' csv header;
```

note how long this process took.

after this has been run, clean up after yourself again:

```sql
DELETE FROM train_positions;
```

and then close the `psql` shell session:

```sql
\q
```

## 4.4: `sql` `COPY` command (local)

finally, let's mimic the process of doing a bulk load from a *local* file to a *local* `postgres` server. as I wrote above, this requires a superuser (e.g. user `postgres`) and that the file is on the same computer as the database server. this is exactly the scenario we set up on our `ec2` server.

open a `psql` shell connected to your *local* (to `ec2`) `postgres` server:

```bash
psql -U postgres
```

again, turn on timing:

```sql
\timing
```

and finally, copy the file into the table `train_positions`

```sql
COPY train_positions
FROM '/tmp/train_positions.csv'
WITH (
    FORMAT csv
    , DELIMITER ','
    , NULL ''
    , HEADER TRUE
);
```

note how long this process took.

after this has been run, clean up after yourself again:

```sql
DELETE FROM train_positions;
```

and then close the `psql` shell session:

```sql
\q
```

## 4.5: delivering you results

put the three different processing times you found on this exercise into a `csv` called `bulk_insert_times.csv` which has the following format:

| method                    | time_in_ms |
|---------------------------|------------|
| `psycopg2` batch `INSERT` |            | 
| `\copy`                   |            |
| `COPY`                    |            |

please report all the times you received in `ms` (this is the unit given for `\timing`, as well as the `python` script I wrote).


##### upload `bulk_insert_times.csv` to the `s3` homework bucket *you* own (from a previous homework assignments)

<div style="border: 1px solid lightgrey;">

# exercise 5: terminate your `postgres` `ec2` instance

via the `aws` `ec2` web console, terminate the `ec2` instance we created above for your `postgres` database

<div style="border: 1px solid lightgrey;">

# exercise 6: terminate your `rds` `postgres` instance

via the `aws` `rds` web console, terminate the `rds` instance we created above for your `postgres` database

<div style="border: 1px solid lightgrey;">

# exercise 7: create a `redshift` cluster

if you didn't do it as part of the `redshift` lecture, go back to the lecture and create your own `redshift` cluster

***note***: if you are not free-tier eligible at this time, please reach out to me to talk about it

<div style="border: 1px solid lightgrey;">

# exercise 8: wmata data: `create table` in `redshift`

I've been downloading `wmata` metro train position information every 10 seconds since late 2016. at this point, I have... a *lot* of individual records of train locations. I have put 20,000,000 of those records into a `csv` and shared it via `s3`, and we're going to load them into a `redshift` instance and do some quick querying.

in order to load this `csv` into `redshift` there must be a destination table to load it into. our first step must be to create that table.

## 8.1: understanding the data

the first ten records of this `csv` are:

```csv
carcount,circuitid,destinationstationcode,directionnum,linecode,secondsatlocation,servicetype,trainid,timestamp
8,2009,F11,2,GR,42,Normal,067,2017-02-20 01:16:02.557021
6,1912,F11,2,GR,0,Normal,175,2017-02-20 01:16:02.557021
6,1480,D13,1,OR,0,Normal,182,2017-02-20 01:16:02.557021
6,2786,D13,1,OR,0,Normal,234,2017-02-20 01:16:02.557021
6,1522,D13,1,OR,48,Normal,272,2017-02-20 01:16:02.557021
6,1384,D13,1,OR,5,Normal,342,2017-02-20 01:16:02.557021
8,2890,D13,1,OR,0,Normal,390,2017-02-20 01:16:02.557021
6,1710,K08,2,OR,615,Normal,061,2017-02-20 01:16:02.557021
6,1288,K08,2,OR,5,Normal,160,2017-02-20 01:16:02.557021
```

you are welcome to download this information to investigate, but it is not necessary for this assignment. you can download it via

```sh
wget https://s3.amazonaws.com/shared.rzl.gu511.com/train_positions/train_positions.csv
```

typically our first questions when creating a new table would be

1. what are the names of our columns?
1. what are the data types of our columns?
1. are any of them appropriate `distkey` or `sortkey` values? what is our distribution strategy?

### 8.1.1: column names

this one is simple: the last column name -- `timestamp` -- is a reserved keyword in `redshift` / `sql` (it is one of the data types). we could keep it as is, but then [we would need to escape it](https://docs.aws.amazon.com/redshift/latest/dg/r_pg_keywords.html) in queries (`select "timestamp", ...`). I'm lazy and don't want to do that.

so when we create that column, let's rename it to `asoftime`.

### 8.1.2: data types

the data types break down roughly as:

+ `integer`: `carcount`, `circuitid`, `directionnum`, `secondsatlocation`, `trainid`
+ `varchar(4)`: `desintationstationcode`, `linecode`
+ `varchar(100)`: `servicetype`
+ `asoftime`: `timestamp`

*note: the numeric lengths of the `varchar` selections above are not particularly empirical; other values may have been more appropriate.*

### 8.1.3: key choice and distribution strategy

I'll cut to the chase: this batch of records doesn't have a good `distkey` candidate. 

a good `distkey` candidate would have *high cardinality* (many values) and relatively even distribution of the number of records per `distkey` value. having an imballance -- e.g. 75% of records having a value of 0 or `NULL` -- is called **skew** and will lead to certain slices of data and certain workers being much more taxed than others (worse than even distribution!).

for `sortkey`, however, we do have a natural sort value -- our `asoftime` column.

when you `create` this table, do not set a `distkey` but do set `asoftime` as the `sortkey`.

## 8.2: the `create table` statement

write a [`create table`](https://docs.aws.amazon.com/redshift/latest/dg/r_CREATE_TABLE_NEW.html) statement that will create a new table that

1. is named `trainpositions`
1. keeps the same column names as the `csv` except for `timestamp` which is renamed `asoftime`
1. uses the provided data types
1. has no `distkey` and has `asoftime` as a `sortkey`

again, the column names and types should be:

| name | type |
|-|-|
| `carcount` | `INTEGER` |
| `circuitid` | `INTEGER` |
| `destinationstationcode` | `VARCHAR(4)` |
| `directionnum` | `INTEGER` |
| `linecode` | `VARCHAR(4)` |
| `secondsatlocation` | `INTEGER` |
| `servicetype` | `VARCHAR(100)` |
| `trainid` | `INTEGER` |
| `asoftime` | `TIMESTAMP` |

save that statement in a file called `create_trainpositions.sql`

##### upload `create_trainpositions.sql` to your s3 homework submission bucket

<div style="border: 1px solid lightgrey;">

# exercise 9: wmata data: `copy`

now that we have created our `trainpositions` table, we must copy our `csv` data into it. at this stage, the questions shift over to the details about the format of the `csv` we wish to import and the permissions we will need to import it, and subsequently how we can configure those values or behaviors in our `copy` command (this is no different than reading a `csv` into `pandas` or an `R` dataframe).

for now, there are a few things we must tackle:

1. our `csv` file has a comma as a delimiter
1. our `csv` file has a header row that you should ignore
1. our `csv` is publically available on `s3` at `s3://shared.rzl.gu511.com/train_positions/train_positions.csv`
    1. this means that in order to execute our `copy` command, `redshift` will need to know about credentials allowing it to access `s3`
1. our `csv` is in the `us-east-1` region

use [the `redshift` `copy` command documentation](https://docs.aws.amazon.com/redshift/latest/dg/r_COPY.html) to figure out how to specify the above configuration options.

create a `copy` statement that will copy the `csv` into `trainpositions` and save it to a file named `copy_trainpositions.sql`

##### upload `copy_trainpositions.sql` to your `s3` homework submission bucket

<div style="border: 1px solid lightgrey;">

# exercise 10: `merge` two `branch`es with non-overlapping edits to the same file

## 10.1: make a local update to `dspipeline.py`

you, being astute and dilligent, notice that the `dspipeline.py` file we committed last week [has no `usage` instructions](https://gist.github.com/RZachLamberty/32f6d0ec0a69e951693eae50e686c2e3#file-dspipeline-py-L13-L15)

```py
Usage:
    <usage>

"""
```

and you decide to fill that in.

update `dspipeline.py` so that it reads

```py
Usage:
    import dspipeline
    dspipeline.adult_data_demo()

"""
```

## 10.2: update `master`

`add` this change, `commit` it with a message `dspipeline: adding usage instructions`, and `push` to `github`

## 10.3: fetch my new `branch`

after pushing to `master` and checking on `github`, you notice that I have sneakily added my *own* updates to `dspipeline.py` as a new `branch` called `minortweak`.

use `git fetch --all` to create a mirror repository of that `branch`.

*note: this branch will be pushed on Saturday afternoon to make sure all users have had time to update their `github` repos from the previous assignment*

## 10.4: `merge` my changes in with yours

use [`git merge`](https://git-scm.com/docs/git-merge) to `merge` the change that I made on the `minortweak` branch into `master`. if given the opportunity to edit a `commit` `message` go with the provided default value, and then `push` the updated `master` `branch` to `github`

*hint: if you're not sure, read the docs above to figure out which branch you should have checked out and which branch name you should include in your `git merge` call*


##### nothing to submit, we will see your commit on `github`

<div style="border: 1px solid lightgrey;">